In [ ]:
!pip uninstall tensorflow

In [ ]:
!pip install tensorflow==1.15 # 1.15 버전 Tensorflow 설치

# SoftMax Classification (Multinomial classification)

In [3]:
import tensorflow as tf
tf.set_random_seed(777)  # for reproducibility

x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3])
nb_class = 3;

W = tf.Variable(tf.random_normal([4, nb_class], name = "weight"))
b = tf.Variable(tf.random_normal([nb_class]), name = "bias")

# softmax
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

In [4]:
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [5]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        cost_val, _ = sess.run([cost, train], feed_dict = {X:x_data, Y:y_data})
        if step % 1000 == 0:
            print(step, cost_val)

    print('--------------')
    # Testing & One-hot encoding
    a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
    print(a, sess.run(tf.argmax(a, 1)))

    print('--------------')
    b = sess.run(hypothesis, feed_dict = {X: [[1, 3, 4, 3]]})
    print(b, sess.run(tf.arg_max(b, 1)))

    print('--------------')
    c = sess.run(hypothesis, feed_dict = {X: [[1, 1, 0, 1]]})
    print(c, sess.run(tf.arg_max(c, 1)))

    print('--------------')
    all = sess.run(hypothesis, feed_dict = {X: [[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]})
    print(all, sess.run(tf.arg_max(all, 1)))

0 6.926112
1000 0.23280531
2000 0.15216155
3000 0.112435445
4000 0.08892067
5000 0.07343659
6000 0.06249354
7000 0.0543602
8000 0.048083052
9000 0.043094594
10000 0.039036863
--------------
[[4.6538315e-07 9.9999952e-01 3.5441222e-08]] [1]
--------------
[[9.974734e-01 2.451452e-03 7.510649e-05]] [0]
--------------
[[8.988214e-16 5.665233e-07 9.999994e-01]] [2]
--------------
[[4.6538227e-07 9.9999952e-01 3.5441289e-08]
 [9.9747342e-01 2.4514566e-03 7.5106625e-05]
 [8.9882139e-16 5.6652323e-07 9.9999940e-01]] [1 0 2]


# Fency Softmax Classifier using zoo data



In [6]:
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)

xy = np.loadtxt('/content/drive/My Drive/모두의 딥러닝/data-04-zoo.csv', delimiter = ',', dtype = np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

In [8]:
print(x_data.shape, y_data.shape)

(101, 16) (101, 1)


In [9]:
nb_class = 7

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])

# one-hot encoding을 하면 rank가 1 증가함 따라서 원래 형태로 바꾸어줘야됨
Y_one_hot = tf.one_hot(Y, nb_class) # one-hot encoding 실행
print("one_hot : ", Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_class]) # 형태 재구성
print("reshape one_hot : ", Y_one_hot)

W = tf.Variable(tf.random_normal([16, nb_class]), name="weight")
b = tf.Variable(tf.random_normal([nb_class]), name = "bias")

logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# version 1
#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels = Y_one_hot))

# version 2
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=tf.stop_gradient([Y_one_hot]))) #stop_gradient함수는 학습하는 부분을 나누어 학습하고자 할 때 사용한다.
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.arg_max(hypothesis, 1) # 가설함수의 값에서 가장 큰 값을 얻음
correct_prediction = tf.equal(prediction, tf.arg_max(Y_one_hot, 1)) # 위에서 얻은 값과 실제 값이 같은지 확인 (1은 열을 기준으로 확인)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) # 앞서 얻은 모든 예측 값을 더해서 평균을 계산해 정확도를 계산

one_hot :  Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape one_hot :  Tensor("Reshape:0", shape=(?, 7), dtype=float32)


In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
        _, cost_val, acc_val = sess.run([train, cost, accuracy], feed_dict={X:x_data, Y:y_data})
        if step % 500 == 0:
            print("Step: {:5}\tCost: {:.3f}\tAcc: {:.2%}".format(step, cost_val, acc_val))

    pred = sess.run(prediction, feed_dict={X:x_data})
    for p, y in zip(pred, y_data.flatten()): # flatten함수는 n차원으로 이루어진 데이터를 직렬화한다.
        print("[{}] Prediction : {} True Y : {}".format(p == int(y), p, int(y)))

Step:     0	Cost: 4.144	Acc: 25.74%
Step:   500	Cost: 0.205	Acc: 95.05%
Step:  1000	Cost: 0.112	Acc: 99.01%
Step:  1500	Cost: 0.076	Acc: 100.00%
Step:  2000	Cost: 0.058	Acc: 100.00%
[True] Prediction : 0 True Y : 0
[True] Prediction : 0 True Y : 0
[True] Prediction : 3 True Y : 3
[True] Prediction : 0 True Y : 0
[True] Prediction : 0 True Y : 0
[True] Prediction : 0 True Y : 0
[True] Prediction : 0 True Y : 0
[True] Prediction : 3 True Y : 3
[True] Prediction : 3 True Y : 3
[True] Prediction : 0 True Y : 0
[True] Prediction : 0 True Y : 0
[True] Prediction : 1 True Y : 1
[True] Prediction : 3 True Y : 3
[True] Prediction : 6 True Y : 6
[True] Prediction : 6 True Y : 6
[True] Prediction : 6 True Y : 6
[True] Prediction : 1 True Y : 1
[True] Prediction : 0 True Y : 0
[True] Prediction : 3 True Y : 3
[True] Prediction : 0 True Y : 0
[True] Prediction : 1 True Y : 1
[True] Prediction : 1 True Y : 1
[True] Prediction : 0 True Y : 0
[True] Prediction : 1 True Y : 1
[True] Prediction : 5 True